# Klavis Gmail MCP Sandbox API Example

This notebook demonstrates the core sandbox lifecycle endpoints for **Gmail**:
- **Acquire** a Gmail sandbox instance
- **Get** sandbox details  
- **Initialize** with email messages and drafts
- **Export** sandbox data
- **Reset** to initial state
- **Release** the sandbox

## 1. Setup

In [ ]:
import os
import httpx
from dotenv import load_dotenv

load_dotenv()

BASE_URL = "https://api.klavis.ai"
KLAVIS_API_KEY = os.environ.get("KLAVIS_API_KEY")

headers = {"Authorization": f"Bearer {KLAVIS_API_KEY}"}

SERVER_NAME = "gmail"

## 2. Acquire Gmail Sandbox

`POST /sandbox/gmail` - Acquire an idle Gmail sandbox instance (where we already setup gmail account and auth setup for you)

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.post(
        f"{BASE_URL}/sandbox/{SERVER_NAME}",
        headers=headers
    )

SANDBOX_ID = resp.json()["sandbox_id"]
print(resp.json())

## 3. Get Sandbox Details

`GET /sandbox` - Get sandbox info that you occupied

`GET /sandbox/gmail/{sandbox_id}` - Get sandbox information

In [ ]:
# Get your sandbox
async with httpx.AsyncClient() as client:
    resp = await client.get(
        f"{BASE_URL}/sandbox",
        headers=headers
    )

print(resp.json())

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.get(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}",
        headers=headers
    )

print(resp.json())

## 4. Initialize Gmail Sandbox with Data

`POST /sandbox/gmail/{sandbox_id}/initialize` - Initialize with email messages and drafts

In [ ]:
# ✏️ Change this to your valid email address
TO_EMAIL = "johndoe@klavis.ai"

In [ ]:
# Example Gmail data (messages + drafts)
gmail_data = {
    "messages": [
        {
            "subject": "Weekly Team Update",
            "to": TO_EMAIL,
            "body": "Hi Team,\n\nHere's our weekly update:\n\n1. Project A is on track\n2. Sprint planning next week\n3. New hire starting Monday\n\nBest regards",
            "labels": ["INBOX", "IMPORTANT"]
        },
        {
            "subject": "Project Status Report",
            "to": TO_EMAIL,
            "body": "Dear Client,\n\nI'm writing to update you on the project status. All milestones are being met on schedule.\n\nKey accomplishments this week:\n- Completed Phase 1\n- Initiated Phase 2\n- Stakeholder review meeting scheduled\n\nBest regards",
            "labels": ["INBOX"]
        },
        {
            "subject": "Meeting Notes - Q4 Planning",
            "to": TO_EMAIL,
            "body": "Hi everyone,\n\nHere are the notes from our Q4 planning session:\n\n- Budget allocation approved\n- New initiatives prioritized\n- Timeline established\n\nPlease review and provide feedback by EOW.\n\nThanks",
            "labels": ["INBOX", "CATEGORY_UPDATES"]
        }
    ],
    "drafts": [
        {
            "subject": "Re: Partnership Proposal",
            "to": "partner@example.com",
            "body": "Hi there,\n\nThank you for reaching out regarding the partnership opportunity. I'm very interested in learning more.\n\nCould we schedule a call next week to discuss further?\n\nBest regards"
        },
        {
            "subject": "Vacation Request - December",
            "to": "hr@example.com",
            "cc": "manager@example.com",
            "body": "Dear HR,\n\nI would like to request vacation time from December 20-30.\n\nPlease let me know if you need any additional information.\n\nThank you"
        }
    ]
}

# Initialize the sandbox with the example gmail data
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.post(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}/initialize",
        headers=headers,
        json=gmail_data
    )

print(resp.json())

## 5. Export Gmail Sandbox

`GET /sandbox/gmail/{sandbox_id}/dump` - Export all emails and drafts from the sandbox so you can see the current environment status!

In [ ]:
import json

async with httpx.AsyncClient() as client:
    resp = await client.get(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}/dump",
        headers=headers
    )

print(json.dumps(resp.json(), indent=2))

## 6. Reset Gmail Sandbox

`POST /sandbox/gmail/{sandbox_id}/reset` - Clear all emails and drafts, for re-use this sandbox

In [ ]:
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.post(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}/reset",
        headers=headers
    )

print(resp.json())

## 7. Release Gmail Sandbox

`DELETE /sandbox/gmail/{sandbox_id}` - Release back to idle pool

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.delete(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}",
        headers=headers
    )

print(resp.json())